In [33]:
import json
import itertools
import numpy as np
from scipy import stats
from bokeh.charts import Histogram, show
from bokeh.plotting import figure, show, vplot, gridplot
from bokeh.io import output_notebook
from scipy.stats import norm
from bokeh.models import LinearAxis, Range1d, BoxAnnotation, GridPlot

In [34]:
class Data(object):  # each data object contains a time, distance, and rssi value
    
    def __init__(self):
        self.time = 0
        self.distance = 0
        self.rssi = 0

In [35]:
beacon_num = "0:0"
all_beacons = {}
block_num = 0
plots = []

In [36]:
def plot_block():  # currently plots pdf for beacon 0:1 within certain block
    global block_num
    global all_beacons
    global beacon_num
    global plots
    
    title = "Beacon " + str(beacon_num) + ", Block " + str(block_num)
    sorted_dist = []
    for i in range(len(all_beacons[beacon_num])):
        sorted_dist.append(all_beacons[beacon_num][i].distance)
    (mu, sigma) = norm.fit(sorted_dist)  # best fit of data
    
    # compute ideal values
    #x = np.linspace(min(sorted_dist), max(sorted_dist), len(sorted_dist))
    x = np.linspace(0, 90, 90)
    pdf = norm.pdf(x, mu, sigma)  # theoretical values
    
    #output_notebook()
    
    p = figure(
        title=title,
        tools="pan, box_zoom, wheel_zoom, reset, resize",
        x_axis_label='Distance (meters)', y_axis_label='Frequency',
        plot_width=400, plot_height=200
    )
    
    hist,edges = np.histogram(sorted_dist, bins=90)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
          fill_color="#036564", line_color="#033649")
    
    p.extra_y_ranges = {"pdf": Range1d(start=min(pdf), end=max(pdf))}  # add y axis for pdf
    #p.extra_y_ranges = {"pdf": Range1d(start=0, end=1)}
    p.line(
        x, pdf, line_color="#D95B43",
        line_width=8, alpha=0.7, legend="PDF",
        y_range_name="pdf"
    )
    p.add_layout(LinearAxis(y_range_name="pdf", axis_label="pdf"), 'left')

    plots.append(p)
    
    sorted_dist.clear()
    #show(p)

In [37]:
def get_data(i):
    global beacon_num
    global all_beacons
    global block_num
    first = True
    begin = False
    
    f = open("2_6_2016_1441.txt")  # open text file
    for line in f:
        words = line.split()
        time = int(words[0])  # get time
        if first == True:
            initial_time = time  # set base time
            first = False
        time = (time - initial_time) / 1000  # get elapsed time in seconds
        parsed_json = json.loads(words[1])  # parse json
        if parsed_json.get('marker'):  # check if marker
            if (parsed_json['marker'] == 'begin\n'):
                begin = True  # begin block
                block_num += 1  # increment block number we are on
            elif (parsed_json.get('marker') == 'end\n'):
                old_begin = begin
                begin = False  # end block
                if (old_begin == True):  # ensures that we have reached the first end after a begin
                    plot_block()  # plot pdf for that block
                all_beacons.clear()  # resets all_beacons dictionary for next block
        elif parsed_json.get('node') and begin == True and parsed_json['minor'] == 1:  # check if node and if in block
            beacon_num = str(parsed_json['major']) + ":" + str(parsed_json['minor'])  # beacon identified by major and minor
            rssi = parsed_json['rssi']
            distance = 10**((rssi+60)/-20)  # log-distance pathloss model
            all_beacons.setdefault(beacon_num, []).append(Data())  # each beacon has an array of Data
            dataPos = len(all_beacons[beacon_num]) - 1  # position in array
            all_beacons[beacon_num][dataPos].time = time
            all_beacons[beacon_num][dataPos].distance = distance
            all_beacons[beacon_num][dataPos].rssi = rssi
    f.close()  # close file

In [38]:
def main():
    global plots
    
    get_data(1)
    output_notebook()
    #grid = gridplot([plots])
    grid = gridplot([plots[0:2], plots[2:4], plots[4:6], plots[6:8], plots[8:]])
    show(grid)

In [39]:
if __name__ == "__main__":
    main()

Loading BokehJS ...